# Reader Monad

## Outline

* Incentive for the Reader monad

* Definition of the Reader monad

* Reader monad examples

In this lesson, we will learn about the Reader monad type and how you can use it.

## Incentive for Reader monad

Imagine we have a global state which is represented by a variable that holds multiple data (e.g. a record syntax variable). 

If we have several functions that need to use part of that data and they call each other, we need to pass that variable from one to another.

Below is an example where we read 4 variables from the operating system environment and store them into a variable of type `Environment`. 

We process this data in 4 function calls and the last function writes the processed data to a configuration file.

In [ ]:
import Data.Maybe (fromMaybe)
import System.Environment (lookupEnv)

data Environment = Environment
    { param1 :: String
    , param2 :: String
    , param3 :: String
    , param4 :: String
    }

loadEnv :: IO Environment
loadEnv = do
  p1 <- lookupEnv "PATH"
  p2 <- lookupEnv "SHELL"
  p3 <- lookupEnv "HOSTNAME"
  p4 <- lookupEnv "HOME"
  return $ Environment
    (fromMaybe "" p1)
    (fromMaybe "" p2)
    (fromMaybe "" p3)
    (fromMaybe "" p4)

func1 :: Environment -> IO ()
func1 env = do
    let content = "System path includes: \n" ++ param1 env ++ "\n\n"
    func2 env content

func2 :: Environment -> String -> IO ()
func2 env content = do
    let updatedContent = content ++ "Shell used is: \n" ++ param2 env ++ "\n\n"
    func3 env updatedContent

func3 :: Environment -> String -> IO ()
func3 env content = do
    let updatedContent = content ++ "Hostname is: \n" ++ param3 env ++ "\n\n"
    func4 env updatedContent

func4 :: Environment -> String -> IO ()
func4 env content = do
    let updatedContent = content ++ "Current home directory is: \n" ++ param4 env ++ "\n\n"
        filePath = param4 env ++ "/conf.txt"
    writeFile filePath updatedContent

We can now write the main execution of our program where we read in the environment and process the data with our 4 functions.

In [ ]:
main1 :: IO ()
main1 = do
  env <- loadEnv
  print "Writing configuration to file: conf.txt"
  func1 env

main1

This is a simple example, but as complexity of programs increases you have to pass around a lot of data. 

For this reason the **Reader monad** was created which can store the state in its environment variable. 

Then it can pass this state variable around between different Reader monads that decide which part of the data to use.

## Definition of the Reader monad

Let's first look at the user freindly definition of **Reader** type and then the actuall Haskell2010 definition that uses transformers.

The definition of the Reader type can be written as:
```haskell
newtype Reader e a = Reader { runReader :: e -> a }
```

We see that the `Reader` data constructor holds a function that can be accessed with the name `runReader`.

It takes in a environment variable and returns a variable of type `a`.

We can now create a Monad instance for `Reader e` and not just `Reader`.

This means the type of our environment variable will remain the same as we compose our function with `(>>=)`.
```haskell
(>>=) :: Reader e a        ->
         (a -> Reader e b) ->
         Reader e b
```

The monad instance for `Reader e` would be:
```haskell
instance Monad (Reader e) where 
  return a = Reader $ const a
  m >>= k  = Reader $ \r -> runReader (k (runReader m r)) r
```

The actual Haskell2010 definition of the `Reader` type is defined in terms of the `ReaderT` monad transformer:
```haskell
type Reader r = ReaderT r Identity
```
We will learn more about it when talk about Monad transformers.

A functions that is usefull for working with the Reader monad is `ask` which returns the environment variable of the Reader monad.

The function `local` takes in a function and a reader moand and updates the reader monad by applying the function to the environment variable.

The `ask` and `local` functions are defined in the `MonadReader` type class which is part of the **Control.Monad.Reader** module.

```haskell
class Monad m => MonadReader r m | m -> r where
    ask   :: m r
    ask = reader id

    local :: (r -> r) -> m a -> m a

    reader :: (r -> a) -> m a
    reader f = do
      r <- ask
      return (f r)
    {-# MINIMAL (ask | reader), local #-}
```

To be able to declare a type class with two types you need to use the *MultiParamTypeClasses* language pragma.

If the type `m` which in reality for us will be `(Reader r)`, is defiinng the type `r` we can state this dependency with the `| m -> r` statement.

To be able to use such dependency statements in type class declarations we need to include the *FunctionalDependencies* language pragma.

For the `Reader e` monad the instance of the `MonadReader` type class would be:
```haskell
instance MonadReader e (Reader e) where
    ask = Reader id
    local f m = Reader $ runReader m . f
```

Form the module that contains the `ReaderMonad` type class we can also use the `asks` function. 

It takes in a function and updates the environment variable of the current reader monad by aplying the function to it.
```haskell
asks :: MonadReader r m => (r -> a) -> m a
asks = reader
```

### Simple example

Below is a simple Reader monad example where we use the `ask` and `runReader` functions.

The programm asks the user for its age and then prints our his age and double times his age.

In [ ]:
import Control.Monad.Reader

age :: Reader Int String
age = do
    env <- ask
    return ("Your age is: " ++ (show env))

ageDouble :: Reader Int String
ageDouble = do
    env <- ask
    return ("Two times your age is: " ++ (show $ 2*env))

bothAges :: Reader Int String
bothAges = do
    a <- age
    ad <- ageDouble
    return (a ++ "\n" ++ ad)

main :: IO ()
main = do
    putStrLn "Input your age:"
    myAge <- read <$> getLine
    putStrLn $ (runReader bothAges) myAge

main

We see that when we call the `age` and `ageDouble` Reader monads in the `bothAges` Reader monad, the environment variabe of the `bothAges` Reader monad is passed onto them.

## Reader monad examples

Let's implement the example from the first chapter with the use of the **Reader monad**.

In [ ]:
processEnv :: Reader Environment String
processEnv = do
    conf1 <- func1'
    conf2 <- func2'
    conf3 <- func3'
    conf4 <- func4'
    let conf = conf1 ++ "\n\n" ++
               conf2 ++ "\n\n" ++
               conf3 ++ "\n\n" ++
               conf4 ++ "\n\n"
    return conf
  
func1' :: Reader Environment String
func1' = do
    env <- ask
    let content = "System path includes: \n" ++ param1 env
    return content

func2' :: Reader Environment String
func2' = do
    env <- ask
    let content = "Shell used is: \n" ++ param2 env
    return content

func3' :: Reader Environment String
func3' = do
    env <- ask
    let content = "Hostname is: \n" ++ param3 env
    return content

func4' :: Reader Environment String
func4' = do
    env <- ask
    let content = "Current home directory is: \n" ++ param4 env
    return content

main3 :: IO ()
main3 = do
    env <- loadEnv
    let configuration = runReader processEnv env
        filePath = param4 env ++ "/conf.txt"
    print "Writing configuration to file: conf.txt"
    writeFile filePath configuration

main3

We see that the functions `func1'` to `func4'` get more cleaner as the functions in the first example. 

The `env` variable is passed from the `processEnv` function to all the other Reader monad functions.

In our next example we update the code from the simple example such that we use the `local` function.

In [ ]:
age :: Reader Int String
age = do
    env <- ask
    return ("My age will be: " ++ (show env))

ageDouble :: Reader Int String
ageDouble = do
    env <- ask
    return ("Two times my age will be: " ++ (show $ 2*env))

bothAges :: Reader Int String
bothAges = do
    let age' = local add2Years age
        ageDouble' = local add2Years ageDouble
    a <- age'
    ad <- ageDouble'
    return (a ++ "\n" ++ ad)

add2Years :: Int -> Int
add2Years years = years + 2

main :: IO ()
main = do
    putStrLn "Input your age:"
    myAge <- read <$> getLine
    putStrLn $ (runReader bothAges) myAge

main

Below is another example on how to lookup variables if you have them stored in a list (taken and adapted from HaskellWiki).

The list contains tuples where the first part is the variable name and the second is variable either of type `String` or of type `Template`. 

What we want to show here is that you can also define a function that takes in data and returns a Reader monad. We use the `asks` function.

In [ ]:
data Template = S String | V Template | T Template deriving Show

data Environment = Env {templates::[(String,Template)],
                        variables::[(String,String)]}

-- lookup a variable from the environment
lookupVar :: String -> Environment -> Maybe String
lookupVar name env = lookup name (variables env)

-- lookup a template from the environment
lookupTemplate :: String -> Environment -> Maybe Template
lookupTemplate name env = lookup name (templates env)

-- resolve a template into a string
resolve :: Template -> Reader Environment (String)
resolve (S s)    = return s
resolve (V t)    = do varName  <- resolve t
                      varValue <- asks (lookupVar varName)
                      return $ maybe "" id varValue
resolve (T t)    = do tmplName <- resolve t
                      body     <- asks (lookupTemplate tmplName)
                      return $ maybe "" show body

template1 :: Template
template1 = V (S "varName1")

template2 :: Template
template2 = T (S "varName2")

myEnv :: Environment
myEnv = Env {
      templates = [("varName1",template1),("varName2",template2)],
      variables = [("varName1","value1"),("varName2","value2")]
    }

main :: IO ()
main = do
    print $ runReader (resolve template1) myEnv
    print $ runReader (resolve template2) myEnv

main

## Recap

In this lesson we've discussed:

- the motivation for introducing the Reader monad type 

- the definition of the Reader monad type and a simple example

- examples that use the Reader monad type to solve a problem